# Describe

The Arc in ArcGIS stands for "always right-click". That's how you get information about the stuff you're working with. But Python scripts can't right-click. They don't have a mouse, or even fingers. You still need the same information, but you're going to need to get it a different way. 

The goal of this notebook is to help you use the Describe function to get information you can use in your script.

## Step 1: Set the workspace

There's a lot of folders on your computer. How is ArcPy supposed to know which one you want it to look in? An important part of working with data is setting the workspace, so ArcPy can figure out what you want.

Run the cell below to figure out the current workspace.

In [54]:
arcpy.env.workspace

'C:\\Users\\tra11070\\OneDrive - Esri\\Documents\\PTL\\apps\\learn_python\\data'

The file path of the workspace is the where ArcPy will look for inputs and write outputs by default. Right now it is probably set to the default geodatabase for your project. If nothing showed up, the workspace isn't set.

You can set the workspace to any folder or database. Run the cell below to set the workspace to the Windows Users directory. After you run the cell below, run the cell above again to see the current workspace.

In [55]:
arcpy.env.workspace = r"C:\Users"

There are a couple things you should have noticed about these two cells:

* There is an `r` character in front of the file path string used to set the workspace directory
* That file path string has a  single backslash (`\`)
* ArcPy showed two backslashes (`\\`) in the output

For stupid reasons related to the Windows/Unix holy wars, Windows file path strings are hard to use in Python. You don't need to know the whole sordid history of the debacle, but you should know this:

* Windows file paths have backslashes.
* Python thinks backslashes in strings have a special meaning.
* In the output, Python will often show file paths as having double backslashes. Typically this is fine.
* When putting file path strings in your code, it's best to use copy/paste to avoid mistakes, which will usually end up giving you paths with single backslashes. When using single backslashes, put an `r` in front of the string to stop Python from treating any backslashes as special.

In the cell below, set the workspace to the data folder inside the folder where this notebook is located.

### Key takeaway: You need to set the workspace, but file paths can be a pain

## Step 2: Describe some data

With the workspace set, you don't have to specify a big long file path to your data. ArcPy knows to look inside the workspace.

Run the cell below to get information about the `main.points` feature class in the `learn_python` geodatabase in the workspace. 

Note: if the cell below returns an error, you most likely have set the workspace incorrectly. Edit the cell above to set the workspace to the data folder.

In [57]:
fc = r"learn_python.geodatabase/main.points"
arcpy.Describe(fc)

catalogPath,C:\Users\tra11070\OneDrive - Esri\Documents\PTL\apps\learn_python\data\learn_python.geodatabase/main.points
dataType,FeatureClass
shapeType,Point
hasM,False
hasZ,False
OBJECTID,OID
lat,Integer
lon,Integer
Shape,Geometry
name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326


The `Describe` function gives you much of the same information you would have gotten by right-cicking on a feature class and choosing properties.

In order to actually use the information in our script, though, it needs to be stored. That means assigning the output of the Describe function to a variable.

Run the cell below to store all the information about `main.points` in a variable called `desc`.

In [101]:
fc = r"learn_python.geodatabase\main.points"
desc = arcpy.Describe(fc)

Run the cells below to access some of that information

In [102]:
desc.shapeType

'Point'

In [103]:
desc.hasZ

False

In [104]:
desc.spatialReference.name

'GCS_WGS_1984'

In the cells below, use Describe to output the shape type and spatial reference name of the `main.boundary` feature class in the `learn_python` geodatabase.

### Key takeaway: Assign the Describe output to a variable and access information from that variable.

## Step 3: Use Describe in a Geoprocessing tool

You ever run a geoprocessing tool, and then realized the output was wrong because you misremembered the spatial reference? Then you had to run the tool again, wasting your time and making you feel dumb?

An advantage of ArcPy generally, and the Describe function specifically, is that you don't have to remember that kind of stuff. You just tell your script to figure it out. Then you can automatically pass the information to a parameter of a geoprocessing tool, and never get it wrong again.

Run the cell below to create a random raster with the same spatial reference and extent as the `main.points` feature class

In [114]:
fc = r'learn_python.geodatabase\main.points'
desc = arcpy.Describe(fc)

with arcpy.EnvManager(outputCoordinateSystem=desc.spatialReference):
    arcpy.management.CreateRandomRaster(
        out_path=arcpy.env.workspace,
        out_name="random.tif",
        raster_extent=desc.extent
    )

Another big advantage of writing code that references the values from Describe is that it makes it really easy change inputs. 

Change the code above so that it generates a random raster with the same extent and spatial reference as the `main.boundary` feature class. Hint: you only need to change one word.

### Key takeaway: Describe makes filling out geoprocessing tool parameters easier

## Step 4: Use Describe for flow control

Remembering a piece of information is only one part of the problem. You also have to remember what you're supposed to do with that information. Sometimes you have to make choices, or move around to different parts of the workflow depending on the properties of the data you're working with. In a script, we call that flow control.

For example, maybe you need to reproject a feature into a state plane coordinate system, but only if it's not already in either state plane or UTM. Without ArcPy, you would:

* Right-click the feature class
* Choose properties
* Check the Spatial Reference
* Stop if it was already in state plane or UTM
* Run the project tool otherwise

This is a common pattern called conditional logic (if this, then do that, otherwise do some other thing). In order to write Python scripts that use conditional logic, you need to use the `if` and `else` statements.

Run the cell below to see a simple example of `if` and `else`.

In [119]:
val = 7
if val == 7:
    print("The variable is equal to 7")
else:
    print("The variable is not equal to 7")

The variable is equal to 7


You should notice that when assigning values to variables, you use `=`. This is you telling Python what the variable equals. But when you are testing to see whether a variable is equal to a value you use `==`. This is you asking Python what the variable equals. `val = 7` is a statement. `val == 7` is a question.

Since the answer to that question is `True` in this case, the code under the `if` block gets executed. If the answer had been `False`, then the code under the `else` block would have been executed instead.

Change the code in the cell above so that the statement in the `else` block gets executed. Hint: There are a couple different changes you could make.

Besides `==`, the other basic conditional logic operators are:

* `>` (is the value on the left greater than the value on the right?)
* `<` (is the value on the left less than the value on the right?)
* `>=` (is the value on the left greater than or equal to the value on the right?)
* `<=` (is the value on the left less than or equal to the value on the right?)
* `!=` (is the value on the left not equal to the value on the right?)

You can also combine multiple questions:

* `or` (at least one is True)
* `and` (both are True)

Run the cell below to use conditional logic to project a feature class (or not) depending on its spatial reference

In [142]:
fc = r"learn_python.geodatabase\main.boundary"
desc = arcpy.Describe(fc)
id = desc.spatialReference.factoryCode

if id == 26915 or id == 26992:
    pass
else:
    arcpy.management.Project(
        in_dataset=fc,
        out_dataset="learn_python.geodatabase\projected",
        out_coor_system=arcpy.SpatialReference(26992),
    )

Nothing happened because `main.boundary` is already in one of the acceptable spatial references (`pass` means do nothing). Change the code above to use `main.points` as the input and see what happens.

### Key takeaway: Describe helps you execute branching logic

## Summary

It defeats the purpose of automation if you have to right click on feature classes to figure out the right parameters to type in your geoprocessing functions. Get ArcPy to do it for you with the Describe function.

Using the output from Describe makes it easier to re-use geoprocessing functions, and it can help you write scripts to change their behavior based on the properties of the data you're working with.